In [52]:
class meu_knn:
    def __init__(self,path_dados):
        self.PATH = path_dados
        self.observacoes_nulas = 0
        self.dados = []
        self.colunas = []
    
    def extrai_dados(self):
        with open(self.PATH, 'r') as arquivo:
            for linha in arquivo:
                self.dados.append(linha.strip().split(','))
        self.colunas = self.dados[0]
        self.dados = self.dados[1:]
    
    def converte_dicionario(self):
        self.dados_dicionario = {self.colunas[x]:[self.dados[y][x] for y in range(len(self.dados))] for x in range(len(self.colunas))}
    
    def sanitiza_dados(self):

        # Remover nulos
        indices_nulos = [i for i, valores in enumerate(zip(*self.dados_dicionario.values())) if any(valor is None for valor in valores)]

        self.observacoes_nulas = len(indices_nulos)

        for chave, valores in self.dados_dicionario.items():
            self.dados_dicionario[chave] = [valor for i, valor in enumerate(valores) if i not in indices_nulos]

        # Confere tipagem
        for chave, valores in self.dados_dicionario.items():
            if chave != 'Outcome':
                try:
                    self.dados_dicionario[chave] = [float(x) for x in valores]
                except:
                    # continue
                    self.dados_dicionario[chave] = [str(x) for x in valores]


    def padroniza_dados(self):
        # Processo de padronização
        for chave, valores in self.dados_dicionario.items():
            if type(valores[0]) == float:
                minimo = min(valores)
                maximo = max(valores)

                self.dados_dicionario[chave] = [(x-minimo)/(minimo-maximo) for x in valores]
    
    def converte_dados_lista(self):
        # Transforma os dados em lista de listas
        self.dados_lista = [[self.dados_dicionario[y][x] for y in self.colunas] for x in range(len(list(self.dados_dicionario.values())[0]))]

    def separa_treino_teste(self, fracao_teste=0.3):
        self.dados_treino = self.dados_lista[:-int(fracao_teste*len(self.dados_lista))]
        self.dados_teste = self.dados_lista[-int(fracao_teste*len(self.dados_lista)):]

    def calcula_distancia(self, vetor_1, vetor_2, peso = 2):

        # Calcular os quadrados das diferenças entre elementos dos vetores
        quadrados = [(abs(x2-x1)) ** peso for x1, x2 in zip(vetor_1, vetor_2)]
        
        # Calcular a soma dos quadrados e a raiz quadrada
        distancia = (sum(quadrados))**(1/peso)

        return distancia
    
    
    # Função que retorna a moda de uma lista
    def retorna_moda(self, lista):
        # Inicializar um dicionário para contar as ocorrências de cada valor único na lista
        comparador = {x: lista.count(x) for x in set(lista)}

        # Retornar o valor com a maior contagem (moda)
        return max(comparador.items(), key=lambda x: x[1])[0]

    # Função que calcula as distâncias entre um vetor de teste e os vetores de treino
    def calcula_distancias(self, teste, peso = 2):
        # Criar uma cópia da base de treino
        treino = self.dados_treino.copy()
        distancias = []

        # Calcular a distância entre o vetor de teste e cada vetor de treino
        for i in range(len(treino)):
            dist = self.calcula_distancia(treino[i][:-1], teste, peso = peso)
            distancias.append(dist)

        return distancias

    # Função que retorna os índices dos k elementos mais próximos
    def indice_k_proximos(self, distancias, k=3):
        lista_tuplas = [(indice, valor) for indice, valor in enumerate(distancias)]

        # Ordenar a lista de tuplas pelos valores das distâncias
        lista_tuplas_ordenada = sorted(lista_tuplas, key=lambda x: x[1])

        # Retornar os índices dos k elementos mais próximos
        return [x[0] for x in lista_tuplas_ordenada[:k]]

    # Função principal que retorna a moda dos k vizinhos mais próximos
    def laudo(self, vetor_principal,peso =2, k=3):

        # Calcular as distâncias entre o vetor de teste e os vetores de treino
        distancias = self.calcula_distancias(vetor_principal, peso = peso)

        # Encontrar os índices dos k vizinhos mais próximos
        indices = self.indice_k_proximos(distancias, k)

        # Obter os rótulos dos vizinhos mais próximos
        laudos = [self.dados_treino[x][-1] for x in indices]

        # Retornar a moda dos rótulos
        laudo = self.retorna_moda(laudos)

        return laudo
    
    def executa(self,fracao_teste =0.3):

        self.extrai_dados()
        self.converte_dicionario()
        self.sanitiza_dados()
        self.padroniza_dados()
        self.converte_dados_lista()
        self.separa_treino_teste(fracao_teste)

    # Função que calcula a taxa de acertos
    def calcula_acertos(self, peso=2 , k=3):
        acertos = 0

        # Contar os acertos comparando os rótulos reais com os rótulos previstos
        for i in range(len(self.dados_teste)):
            expeculacao = self.laudo(self.dados_teste[i][:-1],peso=peso,k = k)
            if self.dados_teste[i][-1] == expeculacao:
                acertos += 1
        
        # Calcular a porcentagem de acertos
        pct_acerto = acertos / len(self.dados_teste)

        return  round(100 * pct_acerto,1)
        
    def confusion_matrix(self, peso = 2, k = 3):
        tp, fp, tn, fn = 0, 0, 0, 0
        

        for teste in self.dados_teste:
            ld = self.laudo(teste, peso, k)
            
            if ld == '1':
                if teste[-1] == '1':
                    tp += 1
                else:
                    fp += 1
            else:
                if teste[-1] == '0':
                    tn += 1
                else:
                    fn += 1

        
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp)
        negative_predictive_value = tn / (tn + fn)
        
        print(f"{'Matriz de Confusão':^50}")
        print()
        print(f"{'True v Pred >':<15}|{'Positivo':^10}|{'Negativo':^10}|")
        print("-" * 38)
        print(f"{'Positivo':<15}|{tp:^10}|{fn:^10}|")
        print("-" * 38)
        print(f"{'Negativo':<15}|{fp:^10}|{tn:^10}|")
        print("-" * 38)
        print(f"Sensitivity: {sensitivity:>20.2f}")
        print(f"Specificity: {specificity:>20.2f}")
        print(f"Precision: {precision:>22.2f}")
        print(f"Negative Predictive Value: {negative_predictive_value:>6.2f}")
        print(f"Accurrancy: {accuracy:>21.2f}")

In [53]:
PATH = './diabetes.csv'

In [54]:
knn = meu_knn(PATH)
knn.executa()
resultados = [ ]
for peso in range(2,10):
    for vizinhos in range(3,100):
        resultados.append((peso, vizinhos, knn.calcula_acertos(peso,vizinhos)))
        
dados_optimizados = sorted(resultados, key= lambda x:x[2])[-1]

In [55]:
knn.confusion_matrix(dados_optimizados[0],dados_optimizados[1])

                Matriz de Confusão                

True v Pred >  | Positivo | Negativo |
--------------------------------------
Positivo       |    41    |    38    |
--------------------------------------
Negativo       |    6     |   145    |
--------------------------------------
Sensitivity:                 0.52
Specificity:                 0.96
Precision:                   0.87
Negative Predictive Value:   0.79
Accurrancy:                  0.81
